In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [3]:
indices = pd.MultiIndex.from_frame(pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), usecols=['Time', 'Vehicle_Number_Plate'], parse_dates=['Time'])).reorder_levels([1,0])
area_centers = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'areas.csv'), index_col=0)
demand = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'demand.csv'), index_col=0)
locations = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), parse_dates=['Time'])
actions = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'actions.csv'), parse_dates=['Time'])

In [4]:
demand.index = pd.to_datetime(demand.index, format='%Y-%m-%d %H:%M')
locations.index = pd.MultiIndex.from_frame(locations.loc[:,['Time', 'Vehicle_Number_Plate']])
actionsi = pd.MultiIndex.from_frame(actions.loc[:,['Time', 'Vehicle_Number_Plate']])
actions.index = actionsi
locations.drop(labels=['Time', 'Vehicle_Number_Plate'], axis=1, inplace=True)
actions.drop(labels=['Time', 'Vehicle_Number_Plate'], axis=1, inplace=True)
vehicle_counts = locations.loc[:, locations.columns[locations.columns.str.contains('Zone')]].groupby('Time').sum()

In [5]:
rental = pd.read_csv((Path.cwd().parent / 'data' / 'interim' / 'rental.csv'), index_col=0, parse_dates=['Start_Datetime_Local', 'End_Datetime_Local'])

In [4]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
event_acc = EventAccumulator(str(Path.cwd().parent /'models'/'stage_1'/'lightning_logs'/'version_1'))
event_acc.Reload()
# Show all tags in the log file

In [5]:
print(event_acc.Tags())

{'images': [], 'audio': [], 'histograms': [], 'scalars': ['hp_metric', 'Loss', 'F1 score', 'epoch'], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}


In [13]:
w_times, step_nums, vals = zip(*event_acc.Scalars('F1 score'))

In [21]:
pd.DataFrame([w_times, step_nums, vals], index=['wall_time', 'step', 'F1']).T.to_csv('lstm_f1.csv')

In [ ]:
pd.merge(locations, actions, left_index=True, right_index=True, how='inner')

In [18]:
pd.merge(a,l,how='inner', on=['Time', 'Vehicle_Number_Plate'])

,Time,Vehicle_Number_Plate
0,2020-02-01 00:56:26,CL35192
1,2020-02-01 01:56:26,CJ79052
2,2020-02-01 07:56:26,BY93898
3,2020-02-01 08:26:26,CJ75240
4,2020-02-01 09:56:26,CJ75216
...,...,...
13546,2021-05-03 20:56:26,CY43459
13547,2021-05-03 21:26:26,CT84514
13548,2021-05-03 21:56:26,CX80057
13549,2021-05-03 22:56:26,CL16672


In [10]:
print('Indices')
it = indices.to_list()
%timeit it[1000]
%timeit indices[1000]
print('Locations')
vloc=locations.values
lloc=list(vloc)
tloc=tuple(vloc)
%timeit torch.tensor(locations.loc[indices[1000]].values)
%timeit torch.tensor(locations.loc[it[1000][0], it[1000][1]].values)
%timeit torch.tensor(locations.iloc[1000].values)
%timeit torch.tensor(vloc[1000])
%timeit torch.tensor(lloc[1000])
%timeit torch.tensor(tloc[1000])
print('Actions')
at = actionsi.to_list()
%timeit indices[1000] in actionsi
%timeit indices[1000] in at
%timeit it[1000] in at
print('Demand')
dt = demand.to_dict('index')
%timeit torch.tensor(demand.loc[indices[1000][0]].values)
%timeit torch.tensor(tuple(dt[indices[1000][0]].values()))
%timeit torch.tensor(tuple(dt[it[1000][0]].values()))
print('Counts')
vc = vehicle_counts.to_dict('index')
%timeit torch.tensor(vehicle_counts.loc[indices[1000][0]].values)
%timeit torch.tensor(tuple(vc[indices[1000][0]].values()))
%timeit torch.tensor(tuple(vc[it[1000][0]].values()))

Indices


KeyboardInterrupt: 

In [ ]:
t = len(pd.read_csv(Path.cwd().parent / 'data' / 'processed' / 'locations.csv', usecols=[0]))
o = len(pd.read_csv(Path.cwd().parent / 'data' / 'processed' / 'actions.csv', usecols=[0]))
pos_weight = (t-o)/o
pos_weight